In [57]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
from sklearn.metrics import pairwise_distances
from sklearn.feature_extraction.text import TfidfVectorizer
%matplotlib inline
pd.set_option('display.max_colwidth',999)

### Variable List
df --> DataFrame (1778,10) with original data

df1 --> DataFrame (1612,7) with unused_cols dropped, and null values from Formatted Result dropped

unused_cols -->  DataFrame (1778,3) columns Pukie, Notes and Description

no_work--> DataFrame (63,7) no work was performed - Warmsups and Cooldowns, not needed for performance calcs

lifts --> DataFrame (45,7) lifts only, work completed but no time.  These will be used to calculate volume abilities. 



In [16]:
df = pd.read_csv('data.csv')

In [17]:
df.loc[(df['Work performed'] > 0) & (df['Work time'] > 0),:]

,Date,Workout,Result,Prescribed,Pukie,Work performed,Work time,Formatted Result,Notes,Description
0,2017-04-04,"5 RFT: Power Cleans, Walking Lunges, and Thrusters",3.990000e+05,True,False,42039,399000.0,6 mins 39 secs,Went out too hot,"5 rounds of:\n12 Power Cleans, 65 lbs\n6 Walking Lunges, 65 lbs\n3 Thrusters, 65 lbs"
1,2017-04-02,Cindy,1.113300e+01,True,False,88204,1200000.0,11 rounds + 2 Pull-ups | 332 reps,NaN,20:00 AMRAP:\n5 Pull-ups\n10 Push-ups\n15 Air Squats
2,2017-03-28,21-15-9: Push Press and Box Jumps,4.190000e+05,True,False,21197,419000.0,6 mins 59 secs,"Step ups, got a set of 13 pp on the first round","21-15-9 reps of:\nPush Press, 95 lbs\nBox Jump, 20 in"
5,2017-03-26,"Row : 3x 1 km, rest 3 mins",7.715000e+05,True,False,93741,771500.0,12 mins 51.5 secs,NaN,"Intervals : rest 3 mins\nRow, 1 km | 4:11.9\nRow, 1 km | 4:19.3\nRow, 1 km | 4:20.3"
6,2017-03-24,CrossFit Games Open 17.5,9.820000e+05,True,False,66793,982000.0,16 mins 22 secs,"All thrusters unbroken, 5 of the rounds of dubs unbroken","10 rounds of:\n9 Thrusters, 65 lbs\n35 Double Unders"
7,2017-03-22,Every 1:30 for 10:30: Front Squat,8.572896e+02,True,False,5848,630000.0,1890 lbs,NaN,Every 1:30 for 10:30:\n2 Front Squats | 135 lbs\n2 Front Squats | 135 lbs\n2 Front Squats | 135 lbs\n2 Front Squats | 135 lbs\n2 Front Squats | 135 lbs\n2 Front Squats | 135 lbs\n2 Front Squats | 135 lbs
8,2017-03-22,AMRAP 5 mins: Squat Cleans and AbMat Sit-ups,3.810000e+00,True,False,29387,300000.0,3 rounds + 7 Squat Cleans + 13 AbMat Sit-ups | 104 reps,NaN,"5:00 AMRAP:\n7 Squat Cleans, 75 lbs\n21 AbMat Sit-ups"
9,2017-03-22,AMRAP 5 mins: Hanging Knee Raises and Push Press,3.000000e+00,True,False,2780,300000.0,3 rounds | 84 reps,NaN,"5:00 AMRAP:\n21 Hanging Knee Raises\n7 Push Press, 75 lbs"
10,2017-03-22,AMRAP 5 mins: Wall Balls and Pull-ups,1.600000e+00,True,False,10193,300000.0,1 round + 10 Wall Balls + 3 Pull-ups | 38 reps,Singles on pull ups,"5:00 AMRAP:\n10 Wall Balls, 14 lbs\n15 Pull-ups"
11,2017-03-21,AMRAP 10 mins: Lateral Burpee (Over Barbell)s and Power Snatches,2.833000e+00,True,False,44381,600000.0,2 rounds + 15 Lateral Burpee (Over Barbell)s + 10 Power Snatches | 85 reps,NaN,"10:00 AMRAP:\n15 Lateral Burpee (Over Barbell)s\n15 Power Snatches, 75 lbs"


In [18]:
df_wo = pd.read_csv('df_wo.csv')

In [19]:
unused_cols = df[['Pukie','Notes']]
unused_cols.shape

(1778, 2)

In [20]:
df1 = df.drop(['Pukie','Notes'], axis=1)

In [21]:
df1 = df1.dropna(subset=['Formatted Result'])
df1.shape

(1612, 8)

In [22]:
no_work = df1.loc[(df['Formatted Result'] == 'Completed') & (df['Work performed'] == 0), :]
no_work.shape

(63, 8)

In [23]:
lifts = df1.loc[(df1['Work time'].isnull()) & (df1['Formatted Result'].str.endswith('| Completed') | df1['Formatted Result'].str.startswith('Completed |')), :]

In [24]:
movements = pd.read_csv('movements.csv', index_col='Unnamed: 0')
add_moves = [{'movement':'double-under','frequency':153,'move_class':'Monostructural'},{'movement':'squat','frequency':63,'move_class':'Gymnastics'}]
am = pd.DataFrame(add_moves)
new_move = pd.concat([movements,am], ignore_index=True)


In [25]:
new_move_dict = new_move[['movement','move_class']].to_dict(orient='records')

In [26]:
relate_dict = {}
for row in new_move_dict:
    relate_dict[row['movement']] = row['move_class']

In [27]:
move_dict = movements.movement.to_dict()
res = dict((v,k) for k,v in move_dict.iteritems())

In [28]:
res

{'3 position clean + jerk': 245,
 'ab roller': 33,
 'abmat sit-up': 34,
 'air squat': 35,
 'airdyne bike': 0,
 'airdyne calories': 1,
 'ankle mobility': 36,
 'assault bike': 2,
 'assault bike calories': 3,
 'atlas stone carry': 246,
 'atlas stone clean': 247,
 'atlas stone ground to shoulder': 248,
 'back extension': 37,
 'back rack walking lunge': 249,
 'back squat': 250,
 'back squat 80%': 251,
 'back stroke, swimming': 4,
 'backward roll': 38,
 'ball slam': 252,
 'banded muscle up transition': 39,
 'banded shoulder stretch': 40,
 'bar dip': 41,
 'bar dip (assisted)': 42,
 'bar dip (blue band)': 43,
 'bar facing burpee': 44,
 'barbell carry': 253,
 'barbell complex': 254,
 'barbell curl': 255,
 'barbell rollout': 256,
 'barbell warm-up complex': 45,
 'barrier hop burpee': 46,
 'bear complex': 257,
 'bear crawl': 5,
 'behind the neck push press': 258,
 'bench dip': 47,
 'bench press': 259,
 'bent over barbell row': 260,
 'bike': 6,
 'bike ride (mountain bike)': 7,
 'body row': 48,
 'b

In [29]:
workouts = [x for x in df1.Description]
work_string = ' '.join(workouts)


In [30]:
for i,x in enumerate(workouts):
    print i, x

0 5 rounds of:
12 Power Cleans, 65 lbs
6 Walking Lunges, 65 lbs
3 Thrusters, 65 lbs
1 20:00 AMRAP:
5 Pull-ups
10 Push-ups
15 Air Squats
2 21-15-9 reps of:
Push Press, 95 lbs
Box Jump, 20 in
3 Sets
3 Deadlifts | 205 lbs
3 Deadlifts | 205 lbs
3 Deadlifts | 205 lbs
3 Deadlifts | 205 lbs
4 Sets
1 Split Jerk | 170 lbs
5 Intervals : rest 3 mins
Row, 1 km | 4:11.9
Row, 1 km | 4:19.3
Row, 1 km | 4:20.3
6 10 rounds of:
9 Thrusters, 65 lbs
35 Double Unders
7 Every 1:30 for 10:30:
2 Front Squats | 135 lbs
2 Front Squats | 135 lbs
2 Front Squats | 135 lbs
2 Front Squats | 135 lbs
2 Front Squats | 135 lbs
2 Front Squats | 135 lbs
2 Front Squats | 135 lbs
8 5:00 AMRAP:
7 Squat Cleans, 75 lbs
21 AbMat Sit-ups
9 5:00 AMRAP:
21 Hanging Knee Raises
7 Push Press, 75 lbs
10 5:00 AMRAP:
10 Wall Balls, 14 lbs
15 Pull-ups
11 10:00 AMRAP:
15 Lateral Burpee (Over Barbell)s
15 Power Snatches, 75 lbs
12 Sets
1 Back Squat | 250 lbs
13 5 rounds of:
Run, 200 m
12 Dumbbell Walking Lunges, 25 lbs
12 Dumbbell Thruster

In [31]:
move_in_work = []
for w in workouts:
    moves = []
    for m in new_move.movement:
        if m in w.lower():
            moves.append(m)
    move_in_work.append(moves)
    
bar = [', '.join(x) for x in move_in_work]


In [32]:
df1['Movements'] = bar

In [33]:
df1['power'] = df['Work performed'] / (df['Work time']/1000)

In [34]:
df1 = df1.loc[(df1['Work performed']>1000) & (df1['Work time'] < 5000000),:].copy()

In [35]:
df1.reset_index(drop=True, inplace=True)

In [77]:
df1.loc[df1['Workout'] == 'Cindy', :]

,Date,Workout,Result,Prescribed,Work performed,Work time,Formatted Result,Description,Movements,power
1,2017-04-02,Cindy,11.133,True,88204,1200000.0,11 rounds + 2 Pull-ups | 332 reps,20:00 AMRAP:\n5 Pull-ups\n10 Push-ups\n15 Air Squats,"air squat, pull-up, push-up, squat",73.503333
320,2015-10-16,Cindy,9.467,True,74441,1200000.0,9 rounds + 5 Pull-ups + 4 Push-ups | 279 reps,20:00 AMRAP:\n5 Pull-ups\n10 Push-ups\n15 Air Squats,"air squat, pull-up, push-up, squat",62.034167


In [37]:
def mgw_breakdown(cell):
    mgw = []
    move_list = cell.split(', ')
    for x in move_list:
        if x in relate_dict:
            mgw.append(relate_dict[x][0])
    return ''.join(sorted(set(mgw)))
        

In [38]:
from sklearn.feature_extraction.text import CountVectorizer

In [39]:
X = df1['Movements']

vect = CountVectorizer(vocabulary=res)

X_vect = vect.fit_transform(X)

In [40]:
len(X)

741

In [41]:
X_vect

<741x428 sparse matrix of type '<type 'numpy.int64'>'
	with 887 stored elements in Compressed Sparse Row format>

In [42]:
dtm = pd.DataFrame(X_vect.toarray(), columns = vect.get_feature_names())

In [43]:
dtm

,airdyne bike,airdyne calories,assault bike,assault bike calories,"back stroke, swimming",bear crawl,bike,bike ride (mountain bike),"crawl, swimming",double under,...,weighted push-up,weighted sit-up,weighted step ups holding dumbbell,weighted strict bar dip,weighted strict pull-up,weighted strict ring dip,weighted walking lunge,yoke carry,zercher squat,zercher yoke carry
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [44]:
dtm = pd.concat([df1[['Work performed','Work time','power','Movements','Date','Formatted Result']], dtm], axis=1)
dtm.shape

(741, 434)

In [45]:
dtm['Formatted Result'] = dtm['Formatted Result'].apply(lambda x: 'Time' if re.search(r'(min|sec)', x) else 'Reps')

In [46]:
dtm

,Work performed,Work time,power,Movements,Date,Formatted Result,airdyne bike,airdyne calories,assault bike,assault bike calories,...,weighted push-up,weighted sit-up,weighted step ups holding dumbbell,weighted strict bar dip,weighted strict pull-up,weighted strict ring dip,weighted walking lunge,yoke carry,zercher squat,zercher yoke carry
0,42039,399000.0,105.360902,"lunge, walking lunge, clean, power clean, thruster",2017-04-04,Time,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,88204,1200000.0,73.503333,"air squat, pull-up, push-up, squat",2017-04-02,Reps,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,21197,419000.0,50.589499,"box jump, press, push press",2017-03-28,Time,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,93741,771500.0,121.504861,row,2017-03-26,Time,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,66793,982000.0,68.017312,"double under, thruster",2017-03-24,Time,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,5848,630000.0,9.282540,"front squat, squat",2017-03-22,Reps,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,29387,300000.0,97.956667,"abmat sit-up, sit-up, clean, squat clean, squat",2017-03-22,Reps,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,2780,300000.0,9.266667,"hanging knee raise, press, push press",2017-03-22,Reps,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,10193,300000.0,33.976667,"pull-up, wall ball",2017-03-22,Reps,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,44381,600000.0,73.968333,"burpee, lateral burpee (over barbell), power snatch, snatch",2017-03-21,Reps,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [47]:
dtm['mgw_breakdown'] = dtm['Movements'].apply(mgw_breakdown)

In [48]:
breakdown = pd.DataFrame(dtm['Movements'].apply(mgw_breakdown)) 
breakdown

,Movements
0,GW
1,G
2,GW
3,M
4,MW
5,GW
6,GW
7,GW
8,GW
9,GW


In [49]:
for let in ['G','M','W']:
    dtm[let] = dtm['mgw_breakdown'].apply(lambda x: 1 if re.search(let,x) else 0)

In [50]:
dtm['Date'] = pd.to_datetime(dtm['Date'])
dtm

,Work performed,Work time,power,Movements,Date,Formatted Result,airdyne bike,airdyne calories,assault bike,assault bike calories,...,weighted strict pull-up,weighted strict ring dip,weighted walking lunge,yoke carry,zercher squat,zercher yoke carry,mgw_breakdown,G,M,W
0,42039,399000.0,105.360902,"lunge, walking lunge, clean, power clean, thruster",2017-04-04,Time,0,0,0,0,...,0,0,0,0,0,0,GW,1,0,1
1,88204,1200000.0,73.503333,"air squat, pull-up, push-up, squat",2017-04-02,Reps,0,0,0,0,...,0,0,0,0,0,0,G,1,0,0
2,21197,419000.0,50.589499,"box jump, press, push press",2017-03-28,Time,0,0,0,0,...,0,0,0,0,0,0,GW,1,0,1
3,93741,771500.0,121.504861,row,2017-03-26,Time,0,0,0,0,...,0,0,0,0,0,0,M,0,1,0
4,66793,982000.0,68.017312,"double under, thruster",2017-03-24,Time,0,0,0,0,...,0,0,0,0,0,0,MW,0,1,1
5,5848,630000.0,9.282540,"front squat, squat",2017-03-22,Reps,0,0,0,0,...,0,0,0,0,0,0,GW,1,0,1
6,29387,300000.0,97.956667,"abmat sit-up, sit-up, clean, squat clean, squat",2017-03-22,Reps,0,0,0,0,...,0,0,0,0,0,0,GW,1,0,1
7,2780,300000.0,9.266667,"hanging knee raise, press, push press",2017-03-22,Reps,0,0,0,0,...,0,0,0,0,0,0,GW,1,0,1
8,10193,300000.0,33.976667,"pull-up, wall ball",2017-03-22,Reps,0,0,0,0,...,0,0,0,0,0,0,GW,1,0,1
9,44381,600000.0,73.968333,"burpee, lateral burpee (over barbell), power snatch, snatch",2017-03-21,Reps,0,0,0,0,...,0,0,0,0,0,0,GW,1,0,1


In [51]:
dtm['month'] = dtm['Date'].dt.month
dtm

,Work performed,Work time,power,Movements,Date,Formatted Result,airdyne bike,airdyne calories,assault bike,assault bike calories,...,weighted strict ring dip,weighted walking lunge,yoke carry,zercher squat,zercher yoke carry,mgw_breakdown,G,M,W,month
0,42039,399000.0,105.360902,"lunge, walking lunge, clean, power clean, thruster",2017-04-04,Time,0,0,0,0,...,0,0,0,0,0,GW,1,0,1,4
1,88204,1200000.0,73.503333,"air squat, pull-up, push-up, squat",2017-04-02,Reps,0,0,0,0,...,0,0,0,0,0,G,1,0,0,4
2,21197,419000.0,50.589499,"box jump, press, push press",2017-03-28,Time,0,0,0,0,...,0,0,0,0,0,GW,1,0,1,3
3,93741,771500.0,121.504861,row,2017-03-26,Time,0,0,0,0,...,0,0,0,0,0,M,0,1,0,3
4,66793,982000.0,68.017312,"double under, thruster",2017-03-24,Time,0,0,0,0,...,0,0,0,0,0,MW,0,1,1,3
5,5848,630000.0,9.282540,"front squat, squat",2017-03-22,Reps,0,0,0,0,...,0,0,0,0,0,GW,1,0,1,3
6,29387,300000.0,97.956667,"abmat sit-up, sit-up, clean, squat clean, squat",2017-03-22,Reps,0,0,0,0,...,0,0,0,0,0,GW,1,0,1,3
7,2780,300000.0,9.266667,"hanging knee raise, press, push press",2017-03-22,Reps,0,0,0,0,...,0,0,0,0,0,GW,1,0,1,3
8,10193,300000.0,33.976667,"pull-up, wall ball",2017-03-22,Reps,0,0,0,0,...,0,0,0,0,0,GW,1,0,1,3
9,44381,600000.0,73.968333,"burpee, lateral burpee (over barbell), power snatch, snatch",2017-03-21,Reps,0,0,0,0,...,0,0,0,0,0,GW,1,0,1,3


In [52]:
dtm = pd.get_dummies(dtm, columns=['mgw_breakdown'])

In [53]:
dtm = dtm.loc[dtm['Work performed'] < 150000, :].copy()

In [54]:
dtm.to_csv('model_wo.csv')

In [59]:
vect = TfidfVectorizer()

tfid_matrix = vect.fit_transform(df1['Description'])

dtm2 = pd.DataFrame(tfid_matrix.toarray(), columns = vect.get_feature_names())
dtm2

In [81]:
tfid_matrix.shape

(741, 380)

In [82]:
des_distance = pairwise_distances(tfid_matrix, metric='cosine')

In [83]:
new_df = pd.DataFrame(des_distance, index=df1['Workout'].values, columns=df1['Workout'].values)

In [114]:
new_df['Annie'].mean(axis=1).sort_values()

Annie                                                                                 0.138940
Annie                                                                                 0.138940
Annie                                                                                 0.138940
Annie                                                                                 0.163343
50-40-30-20-10: Deadlifts, Double Unders and AbMat Sit-ups                            0.239232
Annie                                                                                 0.267290
Annie                                                                                 0.267290
Pull-ups, Kettlebell Swing (Russian)s, Sit-up (abmat)s, Double Unders                 0.476322
Barbra                                                                                0.493951
FT: AbMat Sit-ups, Deadlifts, AbMat Sit-ups, and 2 more                               0.527871
50-35-20: Wall Balls, Pull-ups and Double Unders  

In [ ]:
df2 = df2.loc[df2['Work performed'] < 150000, :].copy()

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
feature_cols = [c for c in df2.columns if c!= 'Work time']
X = df2[feature_cols]
y = df2['Work time']

ss = StandardScaler()
Xs = ss.fit_transform(X)




X_train, X_test, y_train, y_test = train_test_split(Xs, y, test_size =0.2,  random_state=2008)

lr = LinearRegression()
lr.fit(X_train,y_train)


from sklearn.model_selection import cross_val_score

print 'cross_val_score: {}'.format(cross_val_score(lr, X_train, y_train, cv =5).mean())

pred = lr.predict(X_test)

plt.scatter(pred, pred - y_test)
plt.hlines(y = 0, xmin= pred.min(), xmax=pred.max())

from sklearn.metrics import r2_score
print 'R2 score: {}'.format(r2_score(y_test, pred))

In [ ]:
check = ss.transform([59000,0,0,0,1,0,0,0])
lr.predict(check)

In [ ]:
lr.predict(check)

Basic Linear Regression with Train_test_split

In [ ]:
feature_cols = [c for c in dtm.columns if c != 'power' and c != 'Movements' and c != 'Date' and c!= 'Work time']
X = dtm[feature_cols]
y = dtm['Work time']

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

ss = StandardScaler()
Xs = ss.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(Xs, y, random_state=2008)

lr = LinearRegression()
lr.fit(X_train,y_train)


from sklearn.model_selection import cross_val_score

print 'cross_val_score: {}'.format(cross_val_score(lr, X_train, y_train, cv =5).mean())

pred = lr.predict(X_test)

plt.scatter(pred, pred - y_test)
plt.hlines(y = 0, xmin= pred.min(), xmax=pred.max())

from sklearn.metrics import r2_score
print 'R2 score: {}'.format(r2_score(y_test, pred))



Ridge/Lasso/ElasticNet with GridSearch and Standardization

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso, Ridge, ElasticNet
from sklearn.model_selection import GridSearchCV, KFold
import numpy as np

In [ ]:
ss = StandardScaler()
Xs = ss.fit_transform(X)

In [ ]:
kf = KFold(n_splits=10, shuffle=True, random_state=101)
ridge = Ridge()
r_params = {
    'alpha': np.logspace(-1, 5, 10),
    'solver':  ['auto', 'sag'],
    'fit_intercept' : [True,False]
}

In [ ]:
gs = GridSearchCV(ridge, cv=kf, param_grid=r_params, scoring='r2')

In [ ]:
gs.fit(Xs,y)
print gs.best_score_
print gs.best_params_

In [ ]:
lasso = Lasso()
l_params = l_params = {
    'alpha': np.linspace(0,5,20),
    'fit_intercept' : [True,False]
}

In [ ]:
gs = GridSearchCV(lasso, cv=kf, param_grid=l_params, scoring='r2')
gs.fit(Xs,y)
print gs.best_score_
print gs.best_params_

In [ ]:
enet = ElasticNet()
en_params = {
    'alpha' : np.linspace(0,5,20),
    'l1_ratio' : np.linspace(0.01,1.0,25)
}

In [ ]:
gs = GridSearchCV(enet, cv=kf, param_grid=en_params, scoring='r2')
gs.fit(Xs,y)
print gs.best_score_
print gs.best_params_

In [ ]:
from sklearn.kernel_ridge import KernelRidge

In [ ]:
kr = KernelRidge()
kr_params = {
    'alpha': np.logspace(-1, 5, 10)
}

In [ ]:
gs = GridSearchCV(kr, cv=kf, param_grid=kr_params, scoring='r2')
gs.fit(Xs,y)
print gs.best_score_
print gs.best_params_

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor

In [ ]:
gb = GradientBoostingRegressor()
rf = RandomForestRegressor()

In [ ]:
rf_params = {
    'n_estimators': range(10,30,5),
    'max_features': ['auto', 'sqrt', 'log2', None]
}

In [ ]:
gs = GridSearchCV(rf, cv=kf, param_grid=rf_params, scoring='r2')
gs.fit(Xs,y)
print gs.best_score_
print gs.best_params_

In [ ]:
gb_params = {
    'n_estimators': [100],
    'loss': ['ls'],
    'max_depth': range(1,11),
    'min_samples_leaf': [1,3,5],
    'max_features' : ['sqrt', 'auto']}

In [ ]:
gs = GridSearchCV(gb, cv=kf, param_grid=gb_params, scoring='r2')
gs.fit(Xs,y)
print gs.best_score_
print gs.best_params_